# CS5340 Lecture 2:  MLE, MAP and Bayesian Methods#
by Harold Soh (harold@comp.nus.edu.sg)

Graduate TAs: Xie Yaqi and Abdul Fatir Ansari

This notebook is a supplement to Lecture 2 of CS5340: Uncertainty Modeling in AI


In [ ]:
%matplotlib inline

import numpy as np
from scipy import optimize

from numpy.random import randn
from numpy import log, exp

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import scipy.stats as stats
import scipy.special

# Create some data from our simulated ranger #

In [ ]:
#create some data (this is our simulated sensor)
N = 3
r = 1 # our object is 1m away
np.random.seed(0)

# true parameters
mu_true = 0.1
var_true = 0.05

# some noise 
x = np.sqrt(var_true)*np.random.randn(N,1) + mu_true

# our model
y =  r + x


In [ ]:
plt.hist(y, 15)

# Learning the Parameters #
In this section, we'll use the theory learnt in lecture (MLE, Bayesian, and MAP) to learn the parameters of the model.

## Maximum Likelihood Estimation (MLE) ##
First, we'll derive the maximum likelihood estimates for $\mu$ and $\sigma^2$ using the closed-form equations developed in class.

In [ ]:
# MLE estimate
# closed form
mu_MLE = np.mean(x)
var_MLE = np.mean( (x)**2 ) - (mu_MLE)**2

In [ ]:
print("mu_MLE = %g, var_MLE = %g"%(mu_MLE, var_MLE))

Let's try using an off-the-shelf optimizer to numerically *minimize* the *negative* log-likehood directly (equivalent to *maximizing* the log likelihood). It should give us the same answer (or something really close)

In [ ]:
# using an optimizer
def negloglike(theta, x):
    mu = theta[0]
    var = exp(theta[1])
    N = x.size
    nll = N*log(var) + np.sum((x - mu)**2/var)
    return nll

In [ ]:
params = optimize.minimize(negloglike, np.array([1, 2]), args=(x), method='BFGS')

In [ ]:
print(params)

mu_MLEopt = params.x[0]
var_MLEopt = exp(params.x[1])

print("--------------------------------------------------------")
print("mu_MLEopt = %g, var_MLEopt = %g"%(mu_MLEopt, var_MLEopt))

We can plot $p(D|\mu, \sigma^2)$ and examine where the MLE is.

In [ ]:
d = 0.01
mp = np.arange(-0.5, 0.5, d)
sp = np.arange(0.01, 0.5, d) #standard deviation
Mp, Sp = np.meshgrid(mp, sp)
Zlike = np.zeros(Mp.shape)
for i in range(Mp.shape[0]):
    for j in range(Mp.shape[1]):
        Zlike[i,j] = exp(np.sum(stats.norm.logpdf(x, loc=Mp[i,j], scale=Sp[i,j])))

def plotDist(M, S, Z, title):
    fig, ax = plt.subplots()
    CS = ax.contourf(M, S**2, Z, 100)
    #ax.clabel(CS, inline=1, fontsize=10)
    ax.set_title(title)
    fig.colorbar(CS, ax=ax)
    ax.set_xlabel('$\mu$')
    ax.set_ylabel('$\sigma^2$')
    return (fig,ax)

plotDist(Mp, Sp, Zlike, 'Likelihood: $p(D|\mu, \sigma^2)$')
plt.scatter(mu_true, var_true, marker='+', color='w')
plt.scatter(mu_MLE, var_MLE, marker='x', color="r")


# Bayesian Approach #
Next, let's try full Bayesian posterior estimation. First, we define the parameters of our prior (a NormalInverseGamma[$\delta, \gamma, \alpha, \beta$]) and plot the prior to get an idea of how it looks like

In [ ]:
# Prior parameters
delta = 0.0 # mean
gamma = 1.0

# put mode of variance at 0.05 
alpha = 5.0
beta = 0.3

mode = beta/(alpha + 1)
mean = beta/(alpha - 1)
print("Mode: %g, Mean: %g"%(mode, mean))

In [ ]:
# plot the prior
Zp = np.zeros(Mp.shape)
for i in range(Mp.shape[0]):
    for j in range(Mp.shape[1]):
        Zp[i,j] = exp(
            stats.norm.logpdf(Mp[i,j], loc=delta, scale=Sp[i,j]/np.sqrt(gamma)) +
            stats.invgamma.logpdf(Sp[i,j]**2, alpha, scale=beta)
        )


In [ ]:
plotDist(Mp, Sp, Zp, 'Prior: $p(\mu, \sigma^2|\delta,\gamma,a,b)$')

Let's plot the *unnormalized posterior* to take a look (this is the numerator $p(D|\mu, \sigma^2)p(\mu, \sigma^2|\delta,\gamma,a,b)$ of the RHS of the Bayes update rule, i.e., before normalizing with $p(D)$).

In [ ]:
Znum = np.zeros(Mp.shape)
for i in range(Mp.shape[0]):
    for j in range(Mp.shape[1]):
        Znum[i,j] = exp(
            np.sum(stats.norm.logpdf(x, loc=Mp[i,j], scale=Sp[i,j])) +
            stats.norm.logpdf(Mp[i,j], loc=delta, scale=Sp[i,j]/np.sqrt(gamma)) +
            stats.invgamma.logpdf(Sp[i,j]**2, alpha, scale=beta)
        )


In [ ]:
plotDist(Mp, Sp, Zlike, 'Likelihood: $p(D|\mu, \sigma^2)$')
plt.scatter(mu_true, var_true, marker='+', color='w')
plt.scatter(mu_MLE, var_MLE, marker='x', color="r")


plotDist(Mp, Sp, Znum, 'Unnormalized posterior: $p(D|\mu, \sigma^2)p(\mu, \sigma^2|\delta,\gamma,a,b)$')
plt.scatter(mu_true, var_true, marker='+', color='w')
plt.scatter(mu_MLE, var_MLE, marker='x', color="r")


We can update the posterior in closed form as described in the lecture

In [ ]:
xbar = np.mean(x)

delta_post = (delta*gamma + N*xbar)/(gamma + N) # mean
gamma_post = gamma + N
alpha_post = alpha + N/2
beta_post = beta + np.sum( (x- xbar)**2)/2 + ((gamma*N)/(gamma + N))*((xbar - delta)**2/2)

In [ ]:

print("Posteriors:\n delta = %g\n gamma = %g\n alpha = %g\n beta = %g\n"%(delta_post, gamma_post, alpha_post, beta_post))

Let's plot this posterior

In [ ]:
Zpost = np.zeros(Mp.shape)
for i in range(Mp.shape[0]):
    for j in range(Mp.shape[1]):
        Zpost[i,j] = exp(
            stats.norm.logpdf(Mp[i,j], loc=delta_post, scale=Sp[i,j]/np.sqrt(gamma_post)) +
            stats.invgamma.logpdf(Sp[i,j]**2, alpha_post, scale=beta_post)
        )


In [ ]:
plotDist(Mp, Sp, Zpost, 'Posterior')
plt.scatter(mu_true, var_true, marker='+', color='w')
plt.scatter(mu_MLE, var_MLE, marker='x', color="r")


## Maximum a Posteriori (MAP) Estimation ##
Finally, let's try MAP estimation which is a "middle ground" approach

In [ ]:
# closed form
mu_MAP = (N*np.mean(x) + gamma*delta)/(N+gamma)  
var_MAP = (np.sum((x - mu_MAP)**2) + 2*beta + gamma*(delta - mu_MAP)**2)/(N + 3 + 2*alpha)

In [ ]:
print("mu_MAP = %g, var_MAP = %g"%(mu_MAP, var_MAP))

Again, we use an optimizer which should give us very similar results to the above

In [ ]:
# using an optimizer
def neglogpost(theta, x):
    mu = theta[0]
    var = exp(theta[1])
    N = x.size
    nlpost = (log(var)*(N+ (2*alpha) + 3)/2) + ((np.sum((x - mu)**2) +  2*beta + (gamma*(delta - mu)**2))/(2*var))
    return nlpost

In [ ]:
neglogpost([mu_MAP, log(var_MAP)], x)

In [ ]:
params = optimize.minimize(neglogpost, np.array([1, 2.0]), args=(x), 
                           method='BFGS'
                          )

In [ ]:
print(params)

mu_MAPopt = params.x[0]
var_MAPopt = exp(params.x[1])

print("mu_MAPopt = %g, var_MAPopt = %g"%(mu_MAPopt, var_MAPopt))

We can also plot the MAP estimate to see where it falls relative to the MLE and Bayesian posterior.

In [ ]:

plotDist(Mp, Sp, Zpost, 'Posterior')
plt.scatter(mu_MLE, var_MLE, marker='x')
plt.scatter(mu_MAP, var_MAP, marker='.')
plt.scatter(mu_true, var_true, marker='+', color='w')


# Making Predictions #

Suppose there is a obstable 2.5m away from our sensor. Since our sensor is imperfect, it returns some reading $\hat{y} = r + x$ where $x$ is drawn from $N(\mu, \sigma^2)$. Let us estimate the real distance of this obstacle based on our earlier computed parameters and this observation.


In [ ]:
r_true = 2.0 # you can change this
X = scipy.stats.norm(loc=mu_true, scale=np.sqrt(var_true))
yhat = r_true + X.rvs(1)
print(yhat)

## MLE and MAP Predictions##
Recall that $X \sim N(\mu, \sigma^2)$. Let the range to the obstacle $R = \hat{y} - X$. R is also normally-distributed, $R \sim N(\hat{y} - \mu, \sigma^2)$. 

In [ ]:
# closed form
r_MLE = yhat - mu_MLE
r_MAP = yhat - mu_MAP

print(" r_MLE = %g \n r_MAP = %g"%(r_MLE, r_MAP))


In [ ]:
z = np.arange(1.0, 3, 0.001)
fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].plot(z, stats.norm.pdf(z, loc=r_MLE, scale=var_MLE), label="MLE")
ax[0].plot(z, stats.norm.pdf(z, loc=r_MAP, scale=var_MAP), label="MAP")
ax[0].axvline(x=r_true, color='red', lw=1)
ax[0].legend()

ax[1].semilogy(z, stats.norm.pdf(z, loc=r_MLE, scale=var_MLE), label="MLE")
ax[1].semilogy(z, stats.norm.pdf(z, loc=r_MAP, scale=var_MAP), label="MAP")
ax[1].axvline(x=r_true, color='red', lw=1)
ax[1].legend()

## Bayesian Predictions ##
The posterior predictive is actually a generalized Student-t distribution. The exact form is given in the lecture slides and on wikipedia: https://en.wikipedia.org/wiki/Student%27s_t-distribution#Generalized_Student's_t-distribution 

The posterior parameters are at: https://en.wikipedia.org/wiki/Conjugate_prior#Continuous_distributions

Given our model, $R$ is also Student-t distributed but with location parameter $\hat{y} - \mu_T$ where $\mu_T$ is the location parameter of our Student-T

In [ ]:
def tpdf(x, delta, gamma, alpha, beta):
    xbar = np.mean(x)
    mu_T = delta
    scale_T = beta*(gamma + 1)/(gamma*alpha)
    dof = 2*alpha
    
    doft = (dof + 1)/2
    
    pref = scipy.special.gamma(doft)/(scipy.special.gamma(dof/2)*np.sqrt(np.pi*dof)*scale_T)
    p = pref*(1 + (1/dof)*((x - mu_T)/scale_T)**2)**(-doft)
    
    return p
    

In [ ]:
z = np.arange(1.0, 3.0, 0.001)
fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].plot(z, stats.norm.pdf(z, loc=r_MLE, scale=var_MLE), label="MLE")
ax[0].plot(z, stats.norm.pdf(z, loc=r_MAP, scale=var_MAP), label="MAP")
ax[0].plot(z, tpdf(z,yhat - delta_post,gamma_post, alpha_post , beta_post), label="Bayes")

ax[0].axvline(x=r_true, color='red', lw=1)
ax[0].legend()

ax[1].semilogy(z, stats.norm.pdf(z, loc=r_MLE, scale=var_MLE), label="MLE")
ax[1].semilogy(z, stats.norm.pdf(z, loc=r_MAP, scale=var_MAP), label="MAP")
ax[1].semilogy(z, tpdf(z,yhat - delta_post,gamma_post, alpha_post , beta_post), label="Bayes")

ax[1].axvline(x=r_true, color='red', lw=1)
ax[1].legend()


If you have set a low number of observations (e.g., $N = 3$), you'll likely see  MAP would typically be overconfident relative to the Bayes estimate (given a broad prior). If you selected a high number of observations, say $N=300$, the estimates should match to a good degree.